In [1]:
import warnings
warnings.filterwarnings(action="ignore")

import pandas as pd
import miceforest as mf
import numpy as np

import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-paper")
import seaborn as sns
import plotly.express as px


# Preprocessing

## Wrangle function

In [14]:
def wrangle(original_file_path):
    # Loading the dataset
    data = pd.read_csv(original_file_path, index_col="id") 

    #--------------------------------------------Feature engineering 0 : Dealing with missing values ---------------------------------------------------#
        ## List of all columns that have more than 40 % missing values proportion
    null_columns = ['sulphurdioxide_so2_column_number_density', 'sulphurdioxide_so2_column_number_density_amf', 'sulphurdioxide_so2_slant_column_number_density', 'sulphurdioxide_cloud_fraction', 'sulphurdioxide_sensor_azimuth_angle', 'sulphurdioxide_sensor_zenith_angle', 'sulphurdioxide_solar_azimuth_angle', 'sulphurdioxide_solar_zenith_angle', 'sulphurdioxide_so2_column_number_density_15km', 'nitrogendioxide_no2_column_number_density', 'nitrogendioxide_tropospheric_no2_column_number_density', 'nitrogendioxide_stratospheric_no2_column_number_density', 'nitrogendioxide_no2_slant_column_number_density', 'nitrogendioxide_tropopause_pressure', 'nitrogendioxide_absorbing_aerosol_index', 'nitrogendioxide_cloud_fraction', 'nitrogendioxide_sensor_altitude', 'nitrogendioxide_sensor_azimuth_angle', 'nitrogendioxide_sensor_zenith_angle', 'nitrogendioxide_solar_azimuth_angle', 'nitrogendioxide_solar_zenith_angle', 'uvaerosollayerheight_aerosol_height', 'uvaerosollayerheight_aerosol_pressure', 'uvaerosollayerheight_aerosol_optical_depth', 'uvaerosollayerheight_sensor_zenith_angle', 'uvaerosollayerheight_sensor_azimuth_angle', 'uvaerosollayerheight_solar_azimuth_angle', 'uvaerosollayerheight_solar_zenith_angle']
    data = data.drop(null_columns, axis = 1)

    #--------------------------------------------Feature engineering 1 : date---------------------------------------------------# 
        ## Creating new variables(month and day) by using the date variable
    data["month"], data["day_of_month"] = data["date"].str.split("-", expand=True)[1], data["date"].str.split("-", expand=True)[2]
        ## Create dict of months
    month_dict = {
        "01":"January",
        "02":"February",
        "03":"March",
        "04":"April",
        "05":"May", 
        "06":"June",
        "07":"July",
        "08":"August",
        "09":"September",
        "10":"October",
        "11":"November",
        "12":"December"
    }
        ## Replace the month by the original values
    data["month"] = data["month"].replace(month_dict)
        ## Lines to determine the day name of each date
            ### Convert the date to another better format
    data["date"] = data["date"].astype(str) + " " + data["hour"].astype(str) + ":00"
    data["date"] = data["date"].apply(lambda x : dt.datetime.strptime(x, "%Y-%m-%d %H:%M"))
            ### Get the day name
    data["day_name"] = data["date"].apply(lambda x : x.strftime("%A"))
        ## Determine wheter it is the weekend or not
    data["is_week_end"] = data["day_name"].apply(lambda day: True if day in ["Saturday", "Sunday"] else False)    

    #--------------------------------------------Feature engineering 2 : lat - long ---------------------------------------------------#
    def get_site_region (latitude):
        if 0 <= latitude < 23.25:
            return "Northern temperal zone"
        elif -23.25 <= latitude<0:
            return "Southern temperal zone"
        elif 23.25 <= latitude < 66.33:
            return "Northerm tropic"
        elif -66.33 <= latitude<-23.25:
            return "Southern tropic"
        elif latitude>= 66.34:
            return "Artic Region"
        elif latitude<= -66.34:
            return "Antartic Region"
    # Create the feature region
    data["region"] = data["site_latitude"].apply(get_site_region)
    #--------------------------------------------Feature engineering 3 : capital lat - long ---------------------------------------------------#
    gc = geonamescache.GeonamesCache()
    countries = gc.get_countries() # Iniate the list of all countries in the world
    cities = gc.get_cities() # Iniate the list of all cities in the world
        ## Function to compute the search for the capital and its coordinates
    def get_capital_coordinates(country_name):
        for country in countries.values():
            if country['name'] == country_name:
                capital_name = country['capital']
                for city_data in cities.values():
                    if city_data['name'] == capital_name:
                        lat, lng = city_data['latitude'], city_data['longitude']
                        return f"{lat}_{lng}"
        ## Creeate a dictionnary to store the capital coordinate associated to each country
    country_capitals_coordinates_dict = {}
    for country in data["country"].unique():
        country_capitals_coordinates_dict[country] = get_capital_coordinates(country)
    
        ## Create the capital coordinates feature in the dataset   
    data[["capital_lat", "capital_long"]] = (
        data["country"].replace(country_capitals_coordinates_dict)
        .str.split("_", expand=True)
        .astype(float)
        )
    #--------------------------------------------Feature engineering 4 : city altitude, population, density ---------------------------------------------------#
        ## Load the population data that we had scrapped
    city_informations = pd.read_csv("Datas/population_data.csv", index_col="City")
        ## Create a function to get the information we need (population, density, altitude)
    def get_city_info(city, info_checked):
        "Take a key to check for the city and return a dict that contains the information related to the city"
        info = city_informations.loc[city][info_checked]
        return info
        ## Call the function to create the features needed
    for info in ["Population", "Density", "Altitude"]:
        data[f"city_{info.lower()}"] = data["city"].apply(lambda row : get_city_info(city=row, info_checked = info))
    #--------------------------------------------Feature engineering 5 : Compute the distance between the sites and the capital ---------------------------------------------------#
        ## Create the function that will commpute the distance with the capital
    def capital_distance(site_longitude, site_latitude, cap_longitude, cap_latitude):
        distance = np.sqrt(
            ((cap_longitude-site_longitude)**2) + ((cap_latitude - site_latitude)**2)
        )
        return distance
        ## Call the previous function a create the distance feature
    data["captial_distance"] = capital_distance(data["site_longitude"], data["site_latitude"], data["capital_long"], data["capital_lat"])
    #--------------------------------------------Feature engineering 7 : Creating meteo features ---------------------------------------------------#
        ## Define a function to find the meteo information for each city for each date
    def get_meteo_info(row):
        location = Point(row['site_latitude'], row['site_longitude'])
        start = row['date']
        end = start

        # Obtenir les données horaires
        data = Hourly(location, start, end)
        meteo_data = data._data  # Assurez-vous que c'est bien le bon attribut

        ## Take from the meteo informations data these values "temp", "dwpt", "rhum", "prcp", "wdir", "wspd", "pres"
        meteo_info = meteo_data[["temp", "dwpt", "rhum", "prcp", "wdir", "wspd", "pres"]].values[0]
        return meteo_info

        ## Call the previous function at each datafram line and create the new column 'meteo_info'
    #data['meteo_info'] = data.apply(get_meteo_info, axis=1, result_type='expand')
    meteo_columns = ["temperature", "dew_point", "relative_humidity", "precipitation", "wind_direction", "wind_speed", "sea-level_air_pressure"]
    data[meteo_columns] = data.apply(lambda row: pd.Series(get_meteo_info(row)), axis=1)
        ## Rename the meteo informations with readable text
    #data[["temperature", "dew_point", "relative_humidity", "precipitation", "wind_direction", "wind_speed", "sea-level_air_pressure"]] = data["meteo_info"].apply(lambda tile : ','.join(str(x) for x in tile)).str.split(",", expand=True)
    data[["temperature", "dew_point", "relative_humidity", "precipitation", "wind_direction", "wind_speed", "sea-level_air_pressure"]] = data[["temperature", "dew_point", "relative_humidity", "precipitation", "wind_direction", "wind_speed", "sea-level_air_pressure"]].astype(float)
    
    #----------------------------""----------------Feature engineering 5 :  ---------------------------------------------------#
    
    #--------------------------------------------Feature engineering 5 :  ---------------------------------------------------#
    
    # Line to delete the columns 
    columns = ["capital_lat", "capital_long", "date", "country", "city"]
    data.drop(columns, axis=1, inplace=True)
    return data


In [15]:
train = wrangle("Datas/Original/Train.csv")
print("-------> Train set informations")
display(train.head(), f'Shape : {train.shape}')


-------> Train set informations


,site_id,site_latitude,site_longitude,city,country,hour,month,carbonmonoxide_co_column_number_density,carbonmonoxide_h2o_column_number_density,carbonmonoxide_cloud_height,...,city_density,city_altitude,captial_distance,temperature,dew_point,relative_humidity,precipitation,wind_direction,wind_speed,sea-level_air_pressure
id,,,,,,,,,,,,,,,,,,,,,
id_vjcx08sz91,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,13,October,NaN,NaN,NaN,...,18232.0,10.0,4.811649,30.0,26.0,79.0,0.5,180.0,16.6,1011.0
id_bkg215syli,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,12,November,0.045475,3771.027210,3399.756845,...,18232.0,10.0,4.811649,33.0,25.8,66.0,0.0,230.0,11.2,1012.4
id_oui2pot3qd,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,13,November,NaN,NaN,NaN,...,18232.0,10.0,4.811649,32.0,25.1,67.0,0.0,210.0,20.5,1010.0
id_9aandqzy4n,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,14,November,NaN,NaN,NaN,...,18232.0,10.0,4.811649,31.0,26.0,75.0,0.5,220.0,13.0,1009.0
id_ali5x2m4iw,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,13,November,0.049045,3514.042054,1678.370478,...,18232.0,10.0,4.811649,31.7,25.6,70.0,0.0,224.0,11.1,1011.1


'Shape : (8071, 65)'

In [16]:
test = wrangle("Datas/Original/Test.csv")
print("-------> Test set informations")
display(test.head(), f'Shape : {test.shape}')

-------> Test set informations


,site_id,site_latitude,site_longitude,city,country,hour,month,carbonmonoxide_co_column_number_density,carbonmonoxide_h2o_column_number_density,carbonmonoxide_cloud_height,...,city_density,city_altitude,captial_distance,temperature,dew_point,relative_humidity,precipitation,wind_direction,wind_speed,sea-level_air_pressure
id,,,,,,,,,,,,,,,,,,,,,
id_ihxgrbq8bw,64f9d17ab9e98d001ac9e882,5.61252,-0.22955,Accra,Ghana,13,September,0.043537,2825.323242,1.000000,...,12386.0,61.0,0.065255,29.2,23.4,71.0,0.2,186.0,20.4,1013.1
id_dg6s4fhiwe,64f9d17ab9e98d001ac9e882,5.61252,-0.22955,Accra,Ghana,13,September,0.036341,2604.788330,1584.809692,...,12386.0,61.0,0.065255,30.0,23.9,70.0,0.2,200.0,19.0,1013.0
id_f7hwwtmuzp,64f9d17ab9e98d001ac9e882,5.61252,-0.22955,Accra,Ghana,13,September,0.037453,3046.314001,90.699029,...,12386.0,61.0,0.065255,31.0,23.1,63.0,0.4,250.0,20.5,1012.0
id_ioese5awdg,64f9d17ab9e98d001ac9e882,5.61252,-0.22955,Accra,Ghana,12,September,NaN,NaN,NaN,...,12386.0,61.0,0.065255,27.0,24.3,85.0,0.3,180.0,22.3,1014.5
id_hdw320zpls,64f9d17ab9e98d001ac9e882,5.61252,-0.22955,Accra,Ghana,12,September,NaN,NaN,NaN,...,12386.0,61.0,0.065255,30.6,24.0,68.0,0.4,180.0,1.8,1012.5


'Shape : (2783, 64)'

In [22]:
train.to_csv("Datas/Train_wrangled.csv")


In [23]:
test.to_csv("Datas/Test_wrangled.csv")

In [25]:
pd.read_csv("Datas/Train_wrangled.csv", index_col="id").shape

(8071, 65)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8071 entries, id_vjcx08sz91 to id_qgxtderh4p
Data columns (total 61 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   site_id                                                   8071 non-null   object        
 1   site_latitude                                             8071 non-null   float64       
 2   site_longitude                                            8071 non-null   float64       
 3   city                                                      8071 non-null   object        
 4   country                                                   8071 non-null   object        
 5   date                                                      8071 non-null   datetime64[ns]
 6   hour                                                      8071 non-null   int64         
 7   month                     

In [14]:
# Line to delete missing values
null_proportions = train.isnull().sum() * 100 / len(train)
null_columns = null_proportions[null_proportions >= 50].index.to_list()

In [69]:
null_proportions.sort_values(ascending=False).head(15)

uvaerosollayerheight_aerosol_height                        94.709454
uvaerosollayerheight_sensor_azimuth_angle                  94.709454
uvaerosollayerheight_aerosol_optical_depth                 94.709454
uvaerosollayerheight_sensor_zenith_angle                   94.709454
uvaerosollayerheight_aerosol_pressure                      94.709454
uvaerosollayerheight_solar_azimuth_angle                   94.709454
uvaerosollayerheight_solar_zenith_angle                    94.709454
nitrogendioxide_sensor_azimuth_angle                       61.541321
nitrogendioxide_solar_zenith_angle                         61.541321
nitrogendioxide_cloud_fraction                             61.541321
nitrogendioxide_absorbing_aerosol_index                    61.541321
nitrogendioxide_tropopause_pressure                        61.541321
nitrogendioxide_no2_slant_column_number_density            61.541321
nitrogendioxide_stratospheric_no2_column_number_density    61.541321
nitrogendioxide_tropospheric_no2_c

In [74]:
(test.isnull().sum() * 100 / len(test)).sort_values(ascending=False).head(25)

cloud_cloud_optical_depth                 45.957600
cloud_cloud_base_pressure                 45.957600
cloud_cloud_fraction                      45.957600
cloud_cloud_top_pressure                  45.957600
cloud_solar_zenith_angle                  45.957600
cloud_solar_azimuth_angle                 45.957600
cloud_sensor_zenith_angle                 45.957600
cloud_sensor_azimuth_angle                45.957600
cloud_surface_albedo                      45.957600
cloud_cloud_top_height                    45.957600
cloud_cloud_base_height                   45.957600
captial_distance                           1.724757
capital_lat                                1.724757
capital_long                               1.724757
ozone_cloud_fraction                       0.503054
ozone_solar_azimuth_angle                  0.503054
ozone_sensor_zenith_angle                  0.503054
ozone_sensor_azimuth_angle                 0.503054
ozone_solar_zenith_angle                   0.503054
ozone_o3_eff

## Dealing with missing values (Imputation)

In [ ]:
def load_wrangle(filepath):
    # Load the database from the filepath
    data = pd.read_csv(filepath, index_col="id")
    # Transform the bool variable
    data["is_week_end"] = data["is_week_end"].astype('category')
    # Transform the object dtype variable into category
    data[data.select_dtypes("object").columns.to_list()] = data[data.select_dtypes("object").columns.to_list()].astype('category')

    # Function to find and imput missing values
    kernel = mf.ImputationKernel(
        data=data,
        datasets=3,  # Create 3 imputed datasets
        save_all_iterations=True,
        random_state=42
        )
    # Perform the imputation
    kernel.mice(3)
    # Retrieve all imputed datasets
    imputed_datasets = [kernel.complete_data(i) for i in range(3)]
        ## Seperate the categorical variables from the numeric variables
    categorical_data =imputed_datasets[0].select_dtypes("category")
        ## Take the numeric variables
    numeric_data = [imputed_datasets[i].select_dtypes("number") for i in range(3)]
    numeric_data = sum(numeric_data) / len(numeric_data) # Compute the mean of imputed values and take it as value to impute
    
    # Delete useless columns/variables
    #numeric_data.drop([column for column in data.columns if "angle" in column or "altitude" in column], axis=1, inplace=True)
    
    # Concatenate the categorical and numeric data
    data = pd.concat([categorical_data, numeric_data], axis=1)
    
    return data